This file loads the TIE-GCM model file. A bunch of challenges with file, as the extension is not .nc but the file is a netcdf. so it hated me for a long time.. this one does work!!!

In [ ]:
import os
import xarray as xr
import numpy as np
import pandas as pd

# Define your model directory
model_directory = './ionosphere_central/CCMC/model_data/TIE-GCM/129/Akshay_Ramesh_042125_IT_5/'

# List all files except .json
model_files = sorted([
    f for f in os.listdir(model_directory)
    if not f.endswith('.json')
])

model_grids, model_times = [], []

for fname in model_files:
    fpath = os.path.join(model_directory, fname)
    try:
        ds = xr.open_dataset(fpath)
        # You may want to check if the variable exists:
        if 'TEC' not in ds.variables:
            print(f"Skipping {fname} (no 'TEC' variable)")
            continue
        tec = ds['TEC'].values  # shape: (time, lat, lon) or (lat, lon) if no time
        # Robust for both with and without time dimension:
        if 'time' in ds:
            times = pd.to_datetime(ds['time'].values)
            for i in range(tec.shape[0]):
                model_grids.append(tec[i] / 1e12)
                model_times.append(times[i])
        else:
            # No time dimension, just add once
            model_grids.append(tec / 1e12)
            model_times.append(None)  # or set to a default/fixed time
    except Exception as e:
        print(f"Failed to load {fname}: {e}")

# Only stack if you got any valid data
if len(model_grids) == 0:
    raise RuntimeError("No valid model grids found! Please check files and data.")

model_grids = np.stack(model_grids)     # (n_times, lat, lon)
model_times = pd.to_datetime(model_times)
# Only keep lats/lons from last file; you can fix this if they might differ!
model_lats = ds['lat'].values
model_lons = ds['lon'].values


Plot example of the TIE-GCM data to check it worked. 

In [ ]:
import cartopy as cpy
import cartopy.crs as ccrs
# plotting TEC data onto a map
import matplotlib.pyplot as plt

tec = ds["TEC"].values
# Create a map projection
projection = ccrs.PlateCarree()
# Create a figure and axis with the projection
fig, ax = plt.subplots(figsize=(10, 5), subplot_kw={'projection': projection})
# Plot the TEC data
# Note: Adjust the extent to your data's lat/lon range
ax.set_extent([-180, 180, -90, 90], crs=projection)
# Add coastlines for reference
ax.coastlines()
# Plot the TEC data
# Note: Adjust the vmin and vmax to your data's range
pcm = ax.pcolormesh(ds["lon"], ds["lat"], tec[1]/1e12, transform=projection, cmap='viridis', vmin=0, vmax=100)
# Add a colorbar
cbar = plt.colorbar(pcm, ax=ax, orientation='vertical', pad=0.05, aspect=50)
cbar.set_label('Total Electron Content (TECU)')
# Set title
ax.set_title('Total Electron Content (TEC) Map')
# Show the plot
plt.show()


Animate TIE-GCM dataset

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import os
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import matplotlib.animation as animation

# --- LOAD MODEL DATA ---
model_directory = './ionosphere_central/CCMC/model_data/TIE-GCM/129/Akshay_Ramesh_042125_IT_5/'
model_files = sorted([f for f in os.listdir(model_directory) if not f.endswith('.json')])

model_grids, model_times = [], []
for fname in model_files:
    fpath = os.path.join(model_directory, fname)
    try:
        ds = xr.open_dataset(fpath)
        if 'TEC' not in ds.variables:
            print(f"Skipping {fname} (no 'TEC' variable)")
            continue
        tec = ds['TEC'].values  # shape: (time, lat, lon) or (lat, lon)
        if 'time' in ds:
            times = pd.to_datetime(ds['time'].values)
            for i in range(tec.shape[0]):
                model_grids.append(tec[i]/1e12)
                model_times.append(times[i])
        else:
            model_grids.append(tec/1e12)
            model_times.append(None)
    except Exception as e:
        print(f"Failed to load {fname}: {e}")

if len(model_grids) == 0:
    raise RuntimeError("No valid model grids found! Please check files and data.")

model_grids = np.stack(model_grids)     # (n_times, lat, lon)
model_times = pd.to_datetime([t if t is not None else pd.NaT for t in model_times])
model_lats = ds['lat'].values
model_lons = ds['lon'].values

# --- ANIMATION OF TIE-GCM MODEL DATA ---
import matplotlib
matplotlib.use('Agg')

# Use robust color scaling
vmin = np.nanmin(model_grids)
vmax = np.nanmax(model_grids)
projection = ccrs.PlateCarree()

fig, ax = plt.subplots(figsize=(12, 6), subplot_kw={'projection': projection})
ax.set_extent([-180, 180, -90, 90], crs=projection)
ax.coastlines()
ax.set_global()

# Initial frame
img = ax.pcolormesh(model_lons, model_lats, model_grids[0], transform=projection,
                    cmap='viridis', vmin=vmin, vmax=vmax, shading='auto')
cb = fig.colorbar(img, ax=ax, orientation='vertical', pad=0.05, aspect=50)
cb.set_label('Total Electron Content (TECU)')

def update(i):
    img.set_array(model_grids[i].ravel())
    tstr = str(model_times[i]) if pd.notnull(model_times[i]) else f"Frame {i}"
    ax.set_title(f'TIE-GCM Total Electron Content (TEC)\nTime: {tstr}')
    return [img]

ani = animation.FuncAnimation(fig, update, frames=model_grids.shape[0], interval=150, blit=False)
ani.save('Yourname_tie_gcm_model_animation.gif', writer='pillow', fps=8)
# --- For Jupyter notebook inline display ---
from IPython.display import HTML
plt.rcParams['animation.embed_limit'] = 50_000_000  # 50 MB (change as needed)
HTML(ani.to_jshtml())
#plt.close(fig)
print(f"Saved TIE-GCM model animation: tie_gcm_model_animation.gif")
